In [1]:
from models import deep_cnn

import os
import numpy as np
import pandas as pd
import random
    
from glob import glob
from scipy.io import wavfile
from scipy.signal import stft

from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from dataset import DatasetGenerator

Using TensorFlow backend.


In [2]:
DIR = 'input' # veri seti konumu

In [3]:
INPUT_SHAPE = (177,98,1)
BATCH = 32 #bir seferde ağda işlenecek veri sayısıdır.
EPOCHS = 15 #veri setinin kaç kere ağdan geçirileceğidir
#hangi kelimelerin eğitileceği. sıralamaya göre id verilir; yes0 no 1 şeklinde.
LABELS = 'yes no'.split()
NUM_CLASSES = len(LABELS)

In [4]:
dsGen = DatasetGenerator(label_set=LABELS)

In [5]:
# veriler okunuyor. label:kelime, label_id:kelimenin idsi, user_id: dosyanın adı, wav_file:dosyanın konumu
df = dsGen.load_data(DIR)

In [6]:
dsGen.apply_train_test_split(test_size=0.3, random_state=2018)
dsGen.apply_train_val_split(val_size=0.2, random_state=2018)

In [7]:
model = deep_cnn(INPUT_SHAPE, NUM_CLASSES)
#Adam default  değerleri (lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc'])

In [9]:
callbacks = [EarlyStopping(monitor='val_acc', patience=4, verbose=1, mode='max')]

In [10]:
history = model.fit_generator(generator=dsGen.generator(BATCH, mode='train'),
                              steps_per_epoch=int(np.ceil(len(dsGen.df_train)/BATCH)),
                              epochs=EPOCHS,
                              verbose=1,
                              callbacks=callbacks,
                              validation_data=dsGen.generator(BATCH, mode='val'),
                              validation_steps=int(np.ceil(len(dsGen.df_val)/BATCH)))

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
84/84 [==============================] - 63s 755ms/step - loss: 0.3068 - acc: 0.8743 - val_loss: 0.2551 - val_acc: 0.8949
Epoch 2/15
84/84 [==============================] - 43s 511ms/step - loss: 0.1486 - acc: 0.9416 - val_loss: 0.7221 - val_acc: 0.7117
Epoch 3/15
84/84 [==============================] - 43s 515ms/step - loss: 0.1493 - acc: 0.9487 - val_loss: 0.2270 - val_acc: 0.9054
Epoch 4/15
84/84 [==============================] - 44s 520ms/step - loss: 0.1863 - acc: 0.9290 - val_loss: 0.4082 - val_acc: 0.8604
Epoch 5/15
84/84 [==============================] - 44s 521ms/step - loss: 0.1307 - acc: 0.9491 - val_loss: 0.1583 - val_acc: 0.9535
Epoch 6/15
84/84 [==============================] - 44s 524ms/step - loss: 0.0956 - acc: 0.9628 - val_loss: 1.0023 - val_acc: 0.6306
Epoch 7/15
84/84 [==============================] - 44s 519ms/step - loss: 0.0664 - acc: 0.9747 - val_loss: 0.4136 - val_acc: 0.8408
Epoch 8/15
84/84 [===

In [ ]:
y_pred_proba = model.predict_generator(dsGen.generator(BATCH, mode='test'),
                                     int(np.ceil(len(dsGen.df_test)/BATCH)),
                                     verbose=1)

In [ ]:
y_pred = np.argmax(y_pred_proba, axis=1)

In [ ]:
y_true = dsGen.df_test['label_id'].values

In [ ]:
dsGen.df_test

In [ ]:
y_true

In [ ]:
acc_score = accuracy_score(y_true, y_pred)

In [ ]:
acc_score

In [ ]:
print("gercek")
for g in y_true:
    print(g,end=", ")
print("\n")
print("tahmin")
for t in y_pred:
    print(t ,end=", ")

print("\n")

print("accuracy(doğruluk):")
print(acc_score)

------------------------------------------------

In [59]:
TEST_DIR = "test_input"

In [70]:
def get_test_data(test_wav_file):
    X_batch = []
    for file in test_wav_file:
        X_batch.append(dsGen.process_wav_file(file))
    X_batch = np.array(X_batch)
    yield X_batch

In [71]:
wav_files = glob(os.path.join(TEST_DIR, '*wav'))

In [72]:
y_pred_proba = model.predict_generator(get_test_data(wav_files),
                                     int(np.ceil(len(dsGen.df_test)/BATCH)),
                                     verbose=1)

1/1 [==============================] - 0s 24ms/step


In [73]:
y_pred = np.argmax(y_pred_proba, axis=1)

In [77]:
y_pred

array([1, 0])

In [75]:
wav_files

['test_input/0bd689d7_nohash_1.wav', 'test_input/0bd689d7_nohash_0.wav']